In [ ]:
from Classifier import Classifier
import numpy as np
import pandas as pd
from dotenv import load_dotenv
import os
from matplotlib import pyplot as plt
import sqlalchemy as sa
import seaborn as sns

color_palette = sns.color_palette()

In [ ]:
load_dotenv()
engine = sa.create_engine(os.getenv("EL_NINO_DB_URL"))

In [ ]:
target_precipitation_df = pd.read_sql('SELECT * FROM target_precipitation;', engine)
target_precipitation_df['date'] = pd.to_datetime(target_precipitation_df['calendar_date'])

sst_weekly_df = pd.read_sql('SELECT * FROM sst_weekly;', engine)
sst_weekly_df['event_calendar_date'] = pd.to_datetime(sst_weekly_df['event_calendar_date'])
sst_weekly_df['yearmonth'] = sst_weekly_df['event_calendar_date'].dt.year.astype(str) + sst_weekly_df['event_calendar_date'].dt.month.astype(str).str.zfill(2)

equatorial_average_temperature_monthly_df = pd.read_sql('SELECT * FROM equatorial_average_temperature_monthly;', engine)
equatorial_average_temperature_monthly_df['event_calendar_date'] = pd.to_datetime(equatorial_average_temperature_monthly_df['event_calendar_date'])

oni_monthly_df = pd.read_sql('SELECT * FROM oni_monthly;', engine)
oni_monthly_df['event_calendar_date'] = pd.to_datetime(oni_monthly_df['event_calendar_date'])

In [ ]:
monthly_df = oni_monthly_df.merge(equatorial_average_temperature_monthly_df, how='outer', on='event_calendar_date')

In [ ]:
monthly_df['yearmonth'] = monthly_df['event_calendar_date'].dt.year.astype(str) + monthly_df['event_calendar_date'].dt.month.astype(str).str.zfill(2)

In [ ]:
X_all = sst_weekly_df.merge(monthly_df, how='outer', left_index=False, right_index=False, on='yearmonth')

In [ ]:
X_all['date'] = X_all['event_calendar_date_x'].combine_first(X_all['event_calendar_date_y'])

In [ ]:
dataset = target_precipitation_df.merge(X_all, how='inner', left_index=False, right_index=False, on='date')
dataset.drop(['event_calendar_date_x', 'event_calendar_date_y', 'yearmonth'], inplace=True, axis=1)
dataset_clean = dataset.dropna(how='any')

In [ ]:
Classifier.classifier(dataset_clean,"target_high_precipitation")